In [1]:
#!pip install mlflow
#!pip install pyarrow --user
#!pip install pandas
#!pip install matplotlib
#!pip install seaborn
#!pip install pycaret --user
#!pip install sklearn --user 

In [2]:
import os
import warnings
import sys

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from sklearn import linear_model, preprocessing, metrics, model_selection
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature
from mlflow.tracking import MlflowClient

import seaborn as sn
import pycaret.classification as pc

if 'inline_rc' not in dir():
    inline_rc = dict(mpl.rcParams)

SEED = 10


# import logging
# logging.basicConfig(level=logging.WARN)
# logger = logging.getLogger(__name__)
np.random.seed(SEED)

# Reset do estilo de cores do matplotlib 

In [3]:
# reset matplotlib

mpl.rcParams.update(inline_rc)
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 14}
mpl.rc('font', **font)
lines = {'linewidth' : 3}
mpl.rc('lines', **lines)

# Sistema Avaliação de Lances

- Carregar dados de vinho e separar vinhos tinto e branco: tintos para desenvolvimento/operação e vinhos brancos para novidade 
- Treinamento de modelo com regressão logística do sklearn no pyCaret
- Registros das etapas de processamento como runs
- Registro do modelo com threshold de precisão mínimo de 70% em Staging
- Aplicação Online: recomendação de vinhos
    - Consumo da base de dados de operação
    - Utilização do requests para fazer o POST e recuperar a predição do modelo
    - Propor os 5 vinhos de alta qualidade de maior nível alcólico (print data frame)
- Aplicação de Monitoramento: pipeline de monitoramento
    - Revalidacao da base de operacao para amostras de controle (simulação especialista)
    - Leitura da base de desenvolvimento (treino+teste)
    - Alarme de amostra de controle
    - Alarme com amostras de novidade


In [4]:
registered_model_name = 'modelo_kobe'
min_precision = 0.7
model_version = -1 # recuperar a ultima versao
nexamples = 4

# Experimento de Classificação de Jogadas

In [5]:
# Para usar o sqlite como repositorio
mlflow.set_tracking_uri("sqlite:///mlruns.db")

experiment_name = 'Projeto Kobe'
experiment = mlflow.get_experiment_by_name(experiment_name)
if experiment is None:
    experiment_id = mlflow.create_experiment(experiment_name)
    experiment = mlflow.get_experiment(experiment_id)
experiment_id = experiment.experiment_id


# Leitura dos Dados de Classificação de Jogadas

In [47]:
df_kobe = pd.read_csv('dataset_kobe.csv',sep=',')


In [52]:
# COLOCAR RUN DE LEITURA DE DADOS
# PARAMETROS: top_features,
# METRICS: SHAPE de cada base de dados
# ARTIFACTS: nenhum

mlflow.end_run()

top_features = ['lat','lon','minutes_remaining','period','playoffs','shot_distance']

with mlflow.start_run(experiment_id=experiment_id, run_name = 'PreparacaoDados'):
    df_kobe = pd.read_csv('dataset_kobe.csv',sep=',')
    df_kobe = df_kobe.dropna()
    
    kobe_target_col = 'shot_made_flag'
    kobe_features_cols = ['','']
    
    #kobe_label_map = df_kobe[[kobe_target_col]].drop_duplicates()
    
    #drop_cols = ['target_label']
    #df_kobe.drop(drop_cols, axis=1, inplace=True)
    
    df_kobe = df_kobe[top_features + ['shot_type', kobe_target_col]].copy()
    data_kobe = df_kobe[df_kobe['shot_type'] == '2PT Field Goal'].copy().drop('shot_type', axis=1)
    
    # Separar parte para compor a base de operacao
    data_kobe, data_operation, ytrain, ytest = model_selection.train_test_split(data_kobe, 
                                                                            data_kobe[kobe_target_col],
                                                                            test_size=0.2)
    data_kobe[kobe_target_col]      = ytrain
    data_operation[kobe_target_col] = ytest

    # Base de kobes brancos
    data_novelty = df_kobe[df_kobe['shot_type'] != '2PT Field Goal'].copy().drop('shot_type', axis=1)

    data_novelty.to_parquet('modelo_kobe_novidade.parquet')
    data_operation.to_parquet('modelo_kobe_operacao.parquet')
    
    # LOG DE PARAMETROS DO MODELO
    mlflow.log_param("top_features", top_features)

    # LOG DE METRICAS GLOBAIS
    mlflow.log_metric("data_train", data_kobe.shape[0])
    mlflow.log_metric("data_test", data_operation.shape[0])
    mlflow.log_metric("data_novelty", data_novelty.shape[0])   
    
mlflow.end_run()

print('== Bases de Dados ==')
print(f'data_kobe {data_kobe.shape}')
print(f'data_operation {data_operation.shape}')
print(f'data_novelty {data_novelty.shape}')
print(f'Columns: {data_kobe.columns}')


== Bases de Dados ==
data_kobe (16228, 7)
data_operation (4057, 7)
data_novelty (5412, 7)
Columns: Index(['lat', 'lon', 'minutes_remaining', 'period', 'playoffs',
       'shot_distance', 'shot_made_flag'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Int64Index: 25697 entries, 1 to 30696
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   lat                25697 non-null  float64
 1   lon                25697 non-null  float64
 2   minutes_remaining  25697 non-null  int64  
 3   period             25697 non-null  int64  
 4   playoffs           25697 non-null  int64  
 5   shot_distance      25697 non-null  int64  
 6   shot_type          25697 non-null  object 
 7   shot_made_flag     25697 non-null  float64
dtypes: float64(3), int64(4), object(1)
memory usage: 1.8+ MB


In [53]:
#!mlflow ui --backend-store-uri sqlite:///mlruns.db

# Treinamento do Modelo

In [54]:
# COLOCAR RUN DE TREINAMENTO DE MODELOS
# PARAMETROS: fold_strategy, fold, model_name, registered_model_name, cross_validation
# METRICS: auto sklearn
# ARTIFACTS: plots

model_name = 'lr'
probability_threshold = 0.5
cross_validation = True
fold_strategy = 'stratifiedkfold',
fold = 10

# train/test
s = pc.setup(data = data_kobe, 
             target = kobe_target_col,
             train_size=0.7,
             silent = True,
             fold_strategy = 'stratifiedkfold',
             fold = fold,
             log_experiment = True, 
             experiment_name = experiment_name, 
             log_plots = True
            )
bestmodel = pc.create_model(model_name,
                            cross_validation = cross_validation, 
                            probability_threshold=probability_threshold)

# Log do run, e nao do modelo respectivo
classification_plots = [ 'auc','pr','confusion_matrix',
#                          'error', 'class_report', 
                        'threshold',
                         'learning','vc','feature',
                       ]
for plot_type in classification_plots:
    print('=> Aplicando plot ', plot_type)
    try:
        artifact = pc.plot_model(bestmodel, plot=plot_type, save=True, use_train_data=False)
        mlflow.log_artifact(artifact)
    except:
        print('=> Nao possivel plotar: ', plot_type )
        continue

pc.save_model(bestmodel, f'./{registered_model_name}') 
# Carrega novamente o pipeline + bestmodel
model_pipe = pc.load_model(f'./{registered_model_name}')


mlflow.end_run()

Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Loaded


# Avaliar Precisão Mínima 

In [45]:
pred_holdout = pc.predict_model(bestmodel)
pred_holdout.info()
pred_holdout['shot_made_flag'] = pred_holdout['shot_made_flag'].astype('int64')
pred_holdout.info()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.5723,0.5957,0.4849,0.5482,0.5146,0.1349,0.1357


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4891 entries, 0 to 4890
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   lat                   4891 non-null   float32
 1   lon                   4891 non-null   float32
 2   shot_distance         4891 non-null   float32
 3   minutes_remaining_0   4891 non-null   float32
 4   minutes_remaining_1   4891 non-null   float32
 5   minutes_remaining_10  4891 non-null   float32
 6   minutes_remaining_11  4891 non-null   float32
 7   minutes_remaining_2   4891 non-null   float32
 8   minutes_remaining_3   4891 non-null   float32
 9   minutes_remaining_4   4891 non-null   float32
 10  minutes_remaining_5   4891 non-null   float32
 11  minutes_remaining_6   4891 non-null   float32
 12  minutes_remaining_7   4891 non-null   float32
 13  minutes_remaining_8   4891 non-null   float32
 14  minutes_remaining_9   4891 non-null   float32
 15  period_1             

ValueError: invalid literal for int() with base 10: '1.0'

In [9]:
# COLOCAR RUN APROVACAO DE MODELO
# PARAMETROS: min_precision
# METRICS: new_version, precision
# ARTIFACTS: None

mlflow.end_run()

with mlflow.start_run(experiment_id=experiment_id, run_name = 'AprovacaoModelo'):
    pred_holdout = pc.predict_model(bestmodel)
    
    pred_holdout['shot_made_flag'] = pred_holdout['shot_made_flag'].astype('int64')
    
    pr = metrics.precision_score(pred_holdout[kobe_target_col], pred_holdout['Label'])
    if pr > min_precision:
        print(f'=> Aceito o modelo com precisão {pr} (min: {min_precision})')
        pred_holdout.to_parquet('modelo_kobe_teste.parquet')
        # Assinatura do Modelo Inferida pelo MLFlow
        model_features = list(data_kobe.drop(kobe_target_col, axis=1).columns)
        inf_signature = infer_signature(data_kobe[model_features], model_pipe.predict(data_kobe))
        # Exemplo de entrada para o MLmodel
        input_example = {x: data_kobe[x].values[:nexamples] for x in model_features}
        # Log do pipeline de modelagem do sklearn e registrar como uma nova versao
        mlflow.sklearn.log_model(
            sk_model=model_pipe,
            artifact_path="sklearn-model",
            registered_model_name=registered_model_name,
            signature = inf_signature,
            input_example = input_example
        )
        # Criacao do cliente do servico MLFlow e atualizacao versao modelo
        client = MlflowClient()
        if model_version == -1:
            model_version = client.get_latest_versions(registered_model_name)[-1].version
        # Registrar o modelo como staging
        client.transition_model_version_stage(
            name=registered_model_name,
            version=model_version, # Verificar com usuario qual versao
            stage="Staging"
        )
    else:
        print(f'=> Rejeitado o modelo com precisão {pr} (min: {min_precision})')

    # LOG DE PARAMETROS DO MODELO
    mlflow.log_param("precisao_minima", min_precision)

    # LOG DE METRICAS GLOBAIS
    mlflow.log_metric("new_version", model_version)
    mlflow.log_metric("precisao", pr)

mlflow.end_run()

NameError: name 'bestmodel' is not defined

# Serviço do Modelo 

In [ ]:
import os
os.environ['MLFLOW_TRACKING_URI'] = 'sqlite:///mlruns.db'
!mlflow models serve -m "models:/modelo_vinho_tinto/Staging" --no-conda -p 5001


# Operação Sistema de Classificação de Jogadas

In [ ]:
import requests
host = 'localhost'
port = '5001'
url = f'http://{host}:{port}/invocations'
headers = {'Content-Type': 'application/json',}

http_data = data_operation.drop(wine_target_col,axis=1).to_json(orient='split')
r = requests.post(url=url, headers=headers, data=http_data)

data_operation.loc[:, 'operation_label'] = pd.read_json(r.text).values[:,0]

data_operation.to_parquet('modelo_kobe_operacao.parquet')
data_operation[data_operation.operation_label == 1].sort_values('alcohol', ascending=False).head(5)

# Revalidacao de Amostras para Monitoramento

In [ ]:
data_control.columns

In [ ]:
# COLOCAR RUN REVALIDACAO
# PARAMETROS: min_samples_control
# METRICS: matriz de confusao
# ARTIFACTS:


# Utilizacao da amostra de controle, que teria sido reavaliada por especialistas
min_samples_control = 150

with mlflow.start_run(experiment_id=experiment_id, run_name = 'RevalidacaoOperacao'):

    data_operation = pd.read_parquet('modelo_vinho_operacao.parquet')

    data_control = data_operation.sample(min_samples_control, random_state=SEED)
    data_control.to_parquet('modelo_vinho_controle.parquet')

    print('== DADOS DE CONTROLE ==')
    print(metrics.classification_report(data_control[wine_target_col], data_control['operation_label']))
    
    # LOG DE PARAMETROS DO MODELO
    mlflow.log_param("min_samples_control", min_samples_control)

    # LOG DE METRICAS GLOBAIS
    cm = metrics.confusion_matrix(data_control[wine_target_col], data_control['operation_label'])
    specificity = cm[0,0] / cm.sum(axis=1)[0]
    sensibility = cm[1,1] / cm.sum(axis=1)[1]
    precision   = cm[1,1] / cm.sum(axis=0)[1]
    mlflow.log_metric("Especificidade", specificity)
    mlflow.log_metric("Sensibilidade", sensibility)
    mlflow.log_metric("Precisao", precision)

mlflow.end_run()

## Alarme de Desvio 

In [ ]:
# reset matplotlib

mpl.rcParams.update(inline_rc)
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 14}
mpl.rc('font', **font)
lines = {'linewidth' : 3}
mpl.rc('lines', **lines)

def data_drift_alarm(var_name, dev_data, data_test, data_control):    
    sn.kdeplot(dev_data[var_name], label='Desenvolvimento')
    sn.kdeplot(data_test[var_name], label='Teste')
    sn.kdeplot(data_control[var_name], label='Monitoramento')
    plt.grid()
    plt.legend(loc='best')
    plt.title(f'Distribuição Variável {var_name}')
    plt.ylabel('Densidade')
    plt.xlabel(f'Unidade de {var_name}')
    plt.tight_layout()

## Alarme de Retreinamento

In [ ]:
def alarm(data_monitoring, testset, min_eff_alarm):
    cm = metrics.confusion_matrix(data_monitoring[wine_target_col], data_monitoring['operation_label'])
    specificity_m = cm[0,0] / cm.sum(axis=1)[0]
    sensibility_m = cm[1,1] / cm.sum(axis=1)[1]
    precision_m   = cm[1,1] / cm.sum(axis=0)[1]

    cm = metrics.confusion_matrix(testset[wine_target_col], testset['Label'])
    specificity_t = cm[0,0] / cm.sum(axis=1)[0]
    sensibility_t = cm[1,1] / cm.sum(axis=1)[1]
    precision_t   = cm[1,1] / cm.sum(axis=0)[1]

    retrain = False
    for name, metric_m, metric_t in zip(['especificidade', 'sensibilidade', 'precisao'],
                                        [specificity_m, sensibility_m, precision_m],
                                        [specificity_t, sensibility_t, precision_t]):
        
        print(f'\t=> {name} de teste {metric_t} e de controle {metric_m}')
        if (metric_t-metric_m)/metric_t > min_eff_alarm:
            print(f'\t=> MODELO OPERANDO FORA DO ESPERADO')
            retrain = True
        else:
            print(f'\t=> MODELO OPERANDO DENTRO DO ESPERADO')
           
        
    return (retrain, [specificity_m, sensibility_m, precision_m],
                                        [specificity_t, sensibility_t, precision_t] ) 

### Monitoramento Base Operacao 

In [ ]:
# COLOCAR RUN MONITORAMENTO OPERACAO
# PARAMETROS: min_eff_alarm
# METRICS: metricas avaliadas e de referencia
# ARTIFACTS:

print('== ALARME DE RETREINAMENTO - BASE CONTROLE ==')
# 10% de desvio aceitavel na metrica. Deve ser estimado pelo conjunto de validacao cruzada. 
min_eff_alarm = 0.1 

with mlflow.start_run(experiment_id=experiment_id, run_name = 'MonitoramentoOperacao'):
    data_control = pd.read_parquet('modelo_vinho_controle.parquet')
    
    (retrain, [specificity_m, sensibility_m, precision_m],
              [specificity_t, sensibility_t, precision_t] ) = alarm(data_control, pred_holdout, min_eff_alarm)
    if retrain:
        print('==> RETREINAMENTO NECESSARIO')
    else:
        print('==> RETREINAMENTO NAO NECESSARIO')
    # LOG DE PARAMETROS DO MODELO
    mlflow.log_param("min_eff_alarm", min_eff_alarm)

    # LOG DE METRICAS GLOBAIS
    mlflow.log_metric("Alarme Retreino", float(retrain))
    mlflow.log_metric("Especificidade Controle", specificity_m)
    mlflow.log_metric("Sensibilidade Controle", sensibility_m)
    mlflow.log_metric("Precisao Controle", precision_m)
    mlflow.log_metric("Especificidade Teste", specificity_t)
    mlflow.log_metric("Sensibilidade Teste", sensibility_t)
    mlflow.log_metric("Precisao Teste", precision_t)
    
    # LOG ARTEFATO
    var_name = 'volatile acidity' # 'alcohol'
    data_drift_alarm(var_name, data_wine, pred_holdout, data_control)
    plot_path = f'monitor_datadrift_{var_name}.png'
    plt.savefig(plot_path)
    mlflow.log_artifact(plot_path)
    

mlflow.end_run()  

# Operação Aplicação Python 

### Monitoramento Base de Novidade

In [ ]:
# COLOCAR RUN MONITORAMENTO NOVIDADE
# PARAMETROS: min_eff_alarm
# METRICS: metricas avaliadas e de referencia
# ARTIFACTS:

print('== ALARME DE RETREINAMENTO - BASE NOVIDADE ==')
# 10% de desvio aceitavel na metrica. Deve ser estimado pelo conjunto de validacao cruzada. 
min_eff_alarm = 0.1 

with mlflow.start_run(experiment_id=experiment_id, run_name = 'MonitoramentoNovidade'):

    model_uri = f"models:/modelo_vinho_tinto/Staging"
    loaded_model = mlflow.sklearn.load_model(model_uri)
    data_novelty = pd.read_parquet('modelo_vinho_novidade.parquet')
    data_novelty.loc[:, 'operation_label'] = loaded_model.predict(data_novelty.drop(wine_target_col,axis=1))
    
    (retrain, [specificity_m, sensibility_m, precision_m],
              [specificity_t, sensibility_t, precision_t] ) = alarm(data_novelty, pred_holdout, min_eff_alarm)
    if retrain:
        print('==> RETREINAMENTO NECESSARIO')
    else:
        print('==> RETREINAMENTO NAO NECESSARIO')
    # LOG DE PARAMETROS DO MODELO
    mlflow.log_param("min_eff_alarm", min_eff_alarm)

    # LOG DE METRICAS GLOBAIS
    mlflow.log_metric("Alarme Retreino", float(retrain))
    mlflow.log_metric("Especificidade Controle", specificity_m)
    mlflow.log_metric("Sensibilidade Controle", sensibility_m)
    mlflow.log_metric("Precisao Controle", precision_m)
    mlflow.log_metric("Especificidade Teste", specificity_t)
    mlflow.log_metric("Sensibilidade Teste", sensibility_t)
    mlflow.log_metric("Precisao Teste", precision_t)
    # LOG ARTEFATO
    var_name = 'volatile acidity' # 'alcohol'
    data_drift_alarm(var_name, data_wine, pred_holdout, data_novelty)
    plot_path = f'novidade_datadrift_{var_name}.png'
    plt.savefig(plot_path)
    mlflow.log_artifact(plot_path)

mlflow.end_run()  